In [1]:
!pip install tokenizers
!pip install transformers
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 8.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.2 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 172 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=cbba2dfc343abec8582f5c46cfd5e6df9e5a9412028133fbaf7f5ce19decd179
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
 

In [27]:
import pandas as pd 
import numpy as np
import time
import datetime
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import sklearn
import bert
import tensorflow_hub as hub
import random
import math
from torch import nn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN, LSTM, GRU, CuDNNLSTM, Embedding
from tensorflow.keras.optimizers import Adam

In [3]:
main_data = pd.read_csv('main_data.csv')
pre_main_data = pd.read_csv('pre_main_data.csv')

In [4]:
main_data.head(20)

,id,label,tweet
0,0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [5]:
labels = main_data.label.unique()
print(labels)

[2 1 0]


In [6]:
pre_main_data.head(20)

,id,label,tweet
0,0,2,! <user> as a woman you should not complain ab...
1,1,1,! <user> boy dats cold tyga dwn bad for cuffin...
2,2,1,! <user> dawg ! <user> you ever fuck a bitch a...
3,3,1,! <user> <user> she look like a tranny
4,4,1,! <user> the shit you hear about me might be t...
5,5,1,! <user> the shit just blows me claim you so f...
6,6,1,! <user> i can not just sit up and hate on ano...
7,7,1,! “ <user> cause i am tired of you big bitches...
8,8,1,and you might not get ya bitch back and thats ...
9,9,1,<user> hobbies include fighting mariam bitch


In [7]:
X = np.array(pre_main_data['tweet'])
Y = np.array(pre_main_data['label'])
print(X.shape , Y.shape)
x, y = sklearn.utils.shuffle(X, Y)

(24433,) (24433,)


# part 2 : Bert and lstm

## Bert


In [8]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [9]:
def tokenize(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [10]:
tokenized = [tokenize(tweet) for tweet in x]

In [11]:
print(tokenized[0])

[1026, 5310, 1028, 1045, 2106, 2025, 4139, 7743, 2229, 1999, 4732, 2082, 2026, 3008, 2071, 2025, 8984, 1996, 1026, 2193, 1028, 4175, 3609, 3482, 2007, 1996, 22147, 3678, 2006, 1996, 2067, 1529]


In [31]:
def shorten_data(token):
  size = len(token)
  final_data = []
  final_size = 30
  for i in range(size):
    temp = token[i]
    if(len(temp))>final_size:
      temp = temp[:final_size]
      final_data.append(np.array(temp))
    else:
      zeros = [0]*(final_size-len(temp))
      temp+=zeros
      final_data.append(np.array(temp))
  return(final_data)
    




In [32]:
data = shorten_data(tokenized)


In [33]:
print(type(data[0]))

<class 'numpy.ndarray'>


#model

In [24]:
def split_Test(Data, Targets):
    length = len(Data)
    num_test=math.floor(length*0.2)
    print(num_test)
    Train=Data[:(length-num_test)]
    targets=Targets[:(length-num_test)]

    Test=Data[(length-num_test):]
    targets_test=Targets[(length-num_test):]
    print(np.shape(Train))
    print(np.shape(Test))
    print(np.shape(targets))
    print(np.shape(targets_test))
    return(Train, Test, targets, targets_test)

In [25]:
x_train, x_test, y_train, y_test = split_Test(data, y)

4886
(19547, 30)
(4886, 30)
(19547,)
(4886,)


In [35]:
top_words = 25000
vector_dimension = 30
lstm_size = 200
max_words = 30
model = Sequential()
model = Sequential()
model.add(LSTM(lstm_size))
model.add(Dense(max_words, activation='relu'))
model.add(Dense(3, activation='softmax'))
adam_1 = Adam(lr=0.008)
model.compile(loss='categorical_crossentropy', optimizer=adam_1,metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [36]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 30)            750000    
                                                                 
 lstm_2 (LSTM)               (None, 200)               184800    
                                                                 
 dense_2 (Dense)             (None, 30)                6030      
                                                                 
 dense_3 (Dense)             (None, 3)                 93        
                                                                 
Total params: 940,923
Trainable params: 940,923
Non-trainable params: 0
_________________________________________________________________
